# Electricity Tariff Analysis
## Setup and Imports

In [11]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load the cleaned electricity tariffs dataset
csv_url = '../data/energy_tariff/median_electricity_tariff.csv'

df = pd.read_csv(csv_url)

print("Raw Dataset Preview:")
print(df.head())
print("\nColumns:", df.columns.tolist())

Raw Dataset Preview:
             Canton Consumption profiles of typical households  Period  \
0            Ticino                                         H4    2020   
1           Grisons                                         C3    2020   
2         Solothurn                                         C3    2024   
3  Basel Landschaft                                         C3    2025   
4               Zug                                         C3    2022   

            Product  Total excl. VAT (Rp./kWH)  Grid usage (Rp./kWH)  \
0  Cheapest product                  20.353333              8.443333   
1  Cheapest product                  18.963333              8.252667   
2  Cheapest product                  30.219436             12.086103   
3  Cheapest product                  27.651200             12.804000   
4  Cheapest product                  18.971711              8.452822   

   Energy supply costs (Rp./kWH)  Community fees (Rp./kWH)  \
0                       7.400000       

## Load and process the data

In [12]:
def process_tariff_data(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Filter for years 2021-2024
    df['Period'] = pd.to_datetime(df['Period'].astype(str), format='%Y')
    df = df[(df['Period'].dt.year >= 2021) & (df['Period'].dt.year <= 2024)]
    
    # Convert prices from Rappen to CHF
    price_columns = [
        'Total excl. VAT (Rp./kWH)',
        'Grid usage (Rp./kWH)',
        'Energy supply costs (Rp./kWH)',
        'Community fees (Rp./kWH)',
        'Feed-in remuneration at cost (KEV) (Rp./kWH)'
    ]
    
    df[price_columns] = df[price_columns].apply(lambda x: x/100)
    
    # Calculate prices before and after renewable surcharge
    df['price_before_surcharge'] = (
        df['Grid usage (Rp./kWH)'] + 
        df['Energy supply costs (Rp./kWH)'] + 
        df['Community fees (Rp./kWH)']
    )
    
    df['renewable_surcharge'] = df['Feed-in remuneration at cost (KEV) (Rp./kWH)']
    df['total_price'] = df['price_before_surcharge'] + df['renewable_surcharge']
    
    # Group by Canton and Year
    df_yearly = df.groupby(['Canton', df['Period'].dt.year]).agg({
        'price_before_surcharge': 'mean',
        'renewable_surcharge': 'mean',
        'total_price': 'mean'
    }).reset_index()
    
    df_yearly.rename(columns={'Period': 'Year'}, inplace=True)
    
    return df_yearly

## Visualize Data

In [13]:
def create_price_trend_plots(df):
    # 1. Create and save static PNG plots
    plt.style.use('seaborn')
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 20))
    
    # Plot 1: Price before renewable surcharge (PNG)
    for canton in df['Canton'].unique():
        canton_data = df[df['Canton'] == canton]
        ax1.plot(canton_data['Year'], canton_data['price_before_surcharge'], marker='o', label=canton)
    ax1.set_title('Energy Price Trend Before Renewable Surcharge by Canton (2021-2024)')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Price (CHF/kWh)')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True)
    
    # Plot 2: Renewable surcharge impact (PNG)
    for canton in df['Canton'].unique():
        canton_data = df[df['Canton'] == canton]
        ax2.plot(canton_data['Year'], canton_data['renewable_surcharge'], marker='o', label=canton)
    ax2.set_title('Renewable Surcharge by Canton (2021-2024)')
    ax2.set_xlabel('Year')
    ax2.set_ylabel('Price (CHF/kWh)')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax2.grid(True)
    
    # Plot 3: Total price trend (PNG)
    for canton in df['Canton'].unique():
        canton_data = df[df['Canton'] == canton]
        ax3.plot(canton_data['Year'], canton_data['total_price'], marker='o', label=canton)
    ax3.set_title('Total Energy Price Trend by Canton (2021-2024)')
    ax3.set_xlabel('Year')
    ax3.set_ylabel('Price (CHF/kWh)')
    ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax3.grid(True)
    
    plt.tight_layout()
    plt.savefig('../visualisations/energy_price_trends.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. Create and save interactive HTML plots
    layout = dict(
        xaxis_title="Year",
        yaxis_title="Price (CHF/kWh)",
        hovermode='x unified',
        template='plotly_white',
        width=1200,
        height=800
    )
    
    # Plot 1: Price before renewable surcharge (HTML)
    fig1 = px.line(df, 
                   x='Year', 
                   y='price_before_surcharge',
                   color='Canton',
                   title='Energy Price Trend Before Renewable Surcharge by Canton (2021-2024)',
                   markers=True)
    fig1.update_layout(**layout)
    fig1.write_html("../visualisations/price_trend_before_surcharge.html")
    
    # Plot 2: Renewable surcharge impact (HTML)
    fig2 = px.line(df,
                   x='Year',
                   y='renewable_surcharge',
                   color='Canton',
                   title='Renewable Surcharge by Canton (2021-2024)',
                   markers=True)
    fig2.update_layout(**layout)
    fig2.write_html("../visualisations/renewable_surcharge_impact.html")
    
    # Plot 3: Total price trend (HTML)
    fig3 = px.line(df,
                   x='Year',
                   y='total_price',
                   color='Canton',
                   title='Total Energy Price Trend by Canton (2021-2024)',
                   markers=True)
    fig3.update_layout(**layout)
    fig3.write_html("../visualisations/total_price_trend.html")
    
    return fig1, fig2, fig3

## Export the data

In [14]:
def export_processed_data(df, output_path):
    # Format dates and round numbers
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    price_columns = ['total_price', 'price_before_surcharge', 'renewable_surcharge']
    df[price_columns] = df[price_columns].round(4)
    
    # Export to CSV
    df.to_csv(output_path, index=False)

## Main execution cell

In [15]:
# Main execution
df_processed = process_tariff_data(csv_url)

# Create visualizations
create_price_trend_plots(df_processed)

# Print summary statistics
print("\nSummary Statistics (2021-2024):")
print("------------------")
print(f"Number of cantons analyzed: {df_processed['Canton'].nunique()}")
print(f"Years covered: 2021-2024")
print("\nMean prices by component (CHF/kWh):")
print(f"Total price: {df_processed['total_price'].mean():.4f}")
print(f"Price before surcharge: {df_processed['price_before_surcharge'].mean():.4f}")
print(f"Renewable surcharge: {df_processed['renewable_surcharge'].mean():.4f}")

# Export the processed dataset
df_processed.to_csv('../data/processed/canton_energy_prices_2021_2024.csv', index=False)


Summary Statistics (2021-2024):
------------------
Number of cantons analyzed: 26
Years covered: 2021-2024

Mean prices by component (CHF/kWh):
Total price: 0.2290
Price before surcharge: 0.2060
Renewable surcharge: 0.0230
